In [2]:
#from HGCalImageAlgo3D import *
from utility_benchmark import *
from timeit import default_timer as timer
from tqdm import tqdm

%matplotlib inline
plt.style.use('classic')
plt.rc("figure",facecolor="w",figsize=(6,4))
plt.rc("font",size=10)
plt.rc("savefig",dpi=300)

In [2]:
DatasetDir  = '/Users/zihengchen/Documents/HGCal/workplace/data/'
DatasetFile = 'CMSSW93X_pt35_pu000'
df    = pd.read_hdf(DatasetDir+"input/"+DatasetFile+"_rechit.h5",key="table")
dfgen = pd.read_pickle(DatasetDir+"input/"+DatasetFile+"_gen.pkl")

FileNotFoundError: File /Users/zihengchen/Documents/HGCal/workplace/data/input/CMSSW93X_pt35_pu000_rechit.h5 does not exist

# 1. Define Fitness

In [4]:
def TranslateDNA(DNA):
    para_upper = np.array([2,8,1])
    para_lower = np.array([0,2,0])
    para = DNA*(para_upper-para_lower)+para_lower
    return para

def DefineFitness(DNA):
    para = TranslateDNA(DNA)
    imagalgo = ImagingAlgo(AFFINITY_Z_ = para[0],
                           KERNAL_R_   = para[1],
                           KERNAL_EXPC_= para[2]
                          )
    
    dfresultclus = imagalgo.RunImagingAlgo(df,
                                           N=100,
                                           verb=False)
    
    gene,cluse,clusn = getoutputclust(DatasetDir,DatasetFile,
                                      N=100,deltarho=5,
                                      dfresultclus=dfresultclus,
                                      dfgen=dfgen)
    
    eff,ave = effsigma(cluse/gene)
    fitness = 1/eff
    return fitness

# 2. Define GA and Optimize

In [5]:
import sys
sys.path.append('../scripts/')
from GeneticAlgo import *

ga = GA(pop_size    = 40,
        DNA_size    = 3, 
        DNA_fitness = DefineFitness,
        cross_rate  = 0.8,
        mutate_rate = 0.01,
        noise_rate  = 0.4,
        noise_amp   = 0.02
       )

In [6]:
fitness_list,DNA_list,best_para_list = [],[],[]
for generation in tqdm(range(50)):
    ga.evolve()
    fitness  = ga.get_pop_fitness()
    DNA      = ga.pop
    best_DNA = ga.pop[np.argmax(fitness)]
    
    fitness_list.append(fitness)
    DNA_list.append(DNA)
    best_para_list.append(TranslateDNA(best_DNA))

100%|██████████| 50/50 [8:45:10<00:00, 607.75s/it]  


In [13]:
fitness_list   = np.array(fitness_list)
DNA_list       = np.array(DNA_list)
best_para_list = np.array(best_para_list)

np.savetxt('../data/parameter_GA/fitness.csv',fitness_list,delimiter=',')
np.savetxt('../data/parameter_GA/DNA.csv',DNA_list,delimiter=',')
#np.savetxt('../data/parameter_GA/best_para.csv',best_para_list,delimiter=',')

TypeError: Mismatch between array dtype ('float64') and format specifier ('%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e')

In [1]:
def TranslateDNA(DNA):
    para_upper = np.array([10])
    para_lower = np.array([0])
    para = DNA*(para_upper-para_lower)+para_lower
    return para
def func(x):
    return -3*np.cos(10*x/np.pi) + x*(10-x) + 4
def DefineFitness(DNA):
    para = TranslateDNA(DNA)
    x = para[0]
    fitness = func(x)
    return fitness

In [14]:
from HGCal_GeneticOptimizer import *
from IPython.display import clear_output
%matplotlib inline

ga = GA(pop_size    = 20,
        DNA_size    = 1, 
        DNA_fitness = DefineFitness,
        cross_rate  = 0.8,
        mutate_rate = 0.05,
        noise_rate  = 0.10,
        noise_amp   = 0.05
       )

In [20]:
fitness_list = []
para_list = []
for generation in range(100):
    print(generation)
    clear_output(wait=True)
    
    ga.evolve()
    
    para_list.append(TranslateDNA(ga.pop))
    fitness  = ga.get_pop_fitness()
    fitness_list.append(fitness)

fitness_list= np.array(fitness_list)
para_list = np.array(para_list)

99


In [16]:
plt.style.use('classic')
plt.style.use('dark_background')
#plt.rc("figure",facecolor="k",figsize=(6,4))
plt.rc("font",size=12)


In [ ]:
for i in range(0,40,2):
    x = np.linspace(0,10,100)
    fig, axes = plt.subplots(2, 1, sharex=False, 
                             gridspec_kw={'height_ratios':[2,1]},
                             figsize=(5,5))
    ax = axes[0]
    
    ax.plot(x,func(x),'w--',lw=2)
    pop = para_list[i]
    ax.scatter(pop,func(pop),edgecolor='none',s=100,color='C0',alpha=0.5)
    ax.set_ylabel('fitness')
    ax.grid(True)
    ax.set_xlim(0,10)
    ax.set_ylim(5,40)
    ax.set_title("generation {}".format(i))
    
    ax = axes[1]
    ax.hist(func(pop),bins=np.arange(0,40,1),histtype="stepfilled",lw=0,label='Fittness')
    #ax.legend()
    ax.set_ylim(0,20)
    ax.grid()
    ax.set_xlabel("Generation Fitness")
    plt.savefig('../plot/generation_{}.png'.format(i),dpi=200)
    plt.close()

#plt.savefig('../plot/generation_{}.png'.format(1))

In [41]:
np.ones([3,2])

array([[ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.]])